# Web scraping

## The goal of this program is to read data from "https://uscar-price.surge.sh/", look for errorneous data, fix them if possible, and store the cleaned data in a '.csv' format. 

## Import necessary libraries

In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import numpy as np
import pickle
import re

## Extract HTML content from the url

In [2]:
url = "https://uscar-price.surge.sh/"

html_content = get(url).content
soup = BeautifulSoup(html_content)

#### Have a look at the html_contents extracted. A more comprehensive look at the contents can be done by viewing page source code on the web browser.

In [3]:
print(soup.prettify()[:3000])

<html>
 <head>
  <title>
   USA-car-price
  </title>
  <style>
   .new-table {
  display: block;
  font-family: sans-serif;
  overflow-x: scroll;
  width: auto;
}


table {
  background-color: #808f7c;
}
  th {
    text-align: center;
    background-color: rgb(119, 176, 175);
    font-weight: normal;
    color: white;
    padding: 20px 30px;

  }
  td {
    background-color: rgb(233, 650, 830);

    /*background-color: #9b9aa1;*/
    padding: 20px 30px;
    color: rgb(111, 111, 111);
  }
body {
    background-color: #fff;
    margin: 0;
    padding: 0;
    font-family: 'Oxygen', sans-serif;
    color: #444;
    font-size: 15px;
    line-height: 18px;
    font-weight: 300;
}

header {
    margin: 0;
    /*background: #505451;*/
    background: linear-gradient(to bottom, #666699 0%, #669999 100%);
    font-family: sans-serif;
    font-style: oblique
    height: 65px;
    padding: 2px;
}

article {
    margin: 40px 30px;
}

h1 {
    color: #fff;
    font-weight: 700;
    font-family: sans

#### Finding tags for columns and data

In [4]:
# From the page source we can uniquely identify the required table belongs to class 'new-table'. So extract it.
cars_table = soup.find('table', attrs={'class':'new-table'})

# Similarly, inside the table, the headers can be found in 'thead' tag.
headings_tr = cars_table.thead.tr

# The data can be found in 'tbody' tag inside the table.
body_tr = cars_table.tbody.find_all('tr')


In [5]:
def return_text(obj):
    if not obj: return None                                     #If obj is None then, return None
    return None if not obj.text.strip() else obj.text.strip()   #Get text from the obj and remove extra spaces, if any, and return the text if not blank, or else return None

#### Store column names in a list and prepare a dictionary to hold the data where the key is the column name.

In [6]:
col_names = []
data = {}

for th in headings_tr.find_all('th'):
    col = return_text(th)
    col_names.append(col)
    if col: data[col] = []

In [7]:
# Fill-in the 'data' dictionary with actual data stored as html in 'body_tr'
for tr in body_tr:
    tds = tr.find_all('td')
    
    for idx,col in enumerate(col_names):
        data[col].append(return_text(tds[idx]))

## Convert extracted data into a DataFrame

In [8]:
df_data = pd.DataFrame(data)

## Explore the data further to look for errorneous data, if any.

In [9]:
# Check if there are any duplicate observations.
df_data.duplicated().sum()

0

In [10]:
df_data.head(8)

,price,brand,model,year,title_status,mileage,color,vin,lot,state,country,condition
0,6300,toyota,cruiser,2008,clean vehicle,274117.0,black,jtezu11f88k007763,159348797,new jersey,usa,10 days left
1,2899,ford,se,2011,clean vehicle,None,silver,2fmdk3gc4bbb02217,166951262,tennessee,usa,6 days left
2,5350,dodge,mpv,2018,clean vehicle,39590.0,silver,3c4pdcgg5jt346413,167655728,georgia,usa,2 days left
3,25000,ford,door,2014,clean vehicle,64146.0,blue,1ftfw1et4efc23745,167753855,virginia,usa,22 hours left
4,27700,chevrolet,1500,2018,clean vehicle,6654.0,red,3gcpcrec2jg473991,167763266,florida,usa,22 hours left
5,5700,dodge,mpv,2018,clean vehicle,45561.0,white,2c4rdgeg9jr237989,167655771,texas,usa,2 days left
6,7300,chevrolet,pk,2010,clean vehicle,149050.0,black,1gcsksea1az121133,167753872,georgia,usa,22 hours left
7,13350,gmc,door,2017,clean vehicle,23525.0,gray,1gks2gkc3hr326762,167692494,california,usa,20 hours left


In [11]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 12 columns):
price           2499 non-null object
brand           2499 non-null object
model           2499 non-null object
year            2499 non-null object
title_status    2499 non-null object
mileage         2309 non-null object
color           2451 non-null object
vin             2499 non-null object
lot             2499 non-null object
state           2499 non-null object
country         2499 non-null object
condition       2499 non-null object
dtypes: object(12)
memory usage: 234.4+ KB


In [12]:
df_data.describe()

,price,brand,model,year,title_status,mileage,color,vin,lot,state,country,condition
count,2499,2499,2499,2499,2499,2309,2451,2499,2499,2499,2499,2499
unique,790,35,127,30,2,2259,49,2495,2495,44,2,47
top,0,ford,door,2019,clean vehicle,1.0,white,3gcrkse37ag234620,167781794,pennsylvania,usa,2 days left
freq,43,1232,651,892,2336,10,697,2,2,299,2492,832


Few observations from 'df_data.head()', 'df_data.info()' and 'df_data.describe()':
1. "brand", "model", "title_status", "color", "vin", "state", "country" and "condition" are text data, and therefore it can be better represented using str data type.
2. "price", "year" and "lot" are integers, and hence can be better represented with int data type. Similarly, "mileage" can be better represented with float data type.
3. There are some null values in "mileage" and "color" columns.
4. vin is unique for most data
5. There are 35 "brands", 127 categories in "model", 2 categories under "title_status", 49 different "color", 44 states and 2 countries. This is a preliminary observation. 
    This may change when we take a deeper look into data.

### Type-cast columns

In [13]:
str_type = ['brand', 'model', 'title_status', 'color', 'vin', 'state', 'country', 'condition']
int_type = ['price', 'year', 'lot']
float_type = ['mileage']

df_data[str_type] = df_data[str_type].astype(str).replace(['None', 'nan'], np.nan)
df_data[int_type] = df_data[int_type].astype(int)
df_data[float_type] = df_data[float_type].astype(float)

## Analyze features individually and perform cleaning, if required

In [14]:
print('brand: ',df_data.brand.unique())

brand:  ['toyota' 'ford' 'dodge' 'chevrolet' 'gmc' 'chrysler' 'kia' 'buick'
 'infiniti' 'mercedes-benz' 'jeep' 'bmw' 'cadillac' 'dogde' 'chervolet'
 'frod' 'hyundai' 'mazda' 'honda' 'geep' 'heartland' 'jaguar' 'acura'
 'harley-davidson' 'hundai' 'audi' 'hinda' 'lincoln' 'lexus' 'nissan'
 'land' 'maserati' 'peterbilt' 'ram' 'nisan']


There are a few spelling errors in "brand" column

### Brand name cleaning

In [15]:
correct_names = ['toyota', 'ford', 'dodge', 'chevrolet', 'gmc', 'chrysler', 'kia', 'buick', 'infiniti', 
                'mercedes-benz', 'jeep', 'bmw', 'cadillac', 'hyundai', 'mazda', 'honda', 'heartland', 
                'jaguar', 'acura', 'harley-davidson', 'audi', 'lincoln', 'lexus', 'nissan', 'land',
                'maserati', 'peterbilt', 'ram']

incorrect_names = [bname for bname in df_data.brand.unique() if not bname in correct_names]

print(incorrect_names)

['dogde', 'chervolet', 'frod', 'geep', 'hundai', 'hinda', 'nisan']


In [16]:
df_data.loc[df_data['brand']=='dogde','brand'] = 'dodge'
df_data.loc[df_data['brand']=='chervolet','brand'] = 'chevrolet'
df_data.loc[df_data['brand']=='frod','brand'] = 'ford'
df_data.loc[df_data['brand']=='geep','brand'] = 'jeep'
df_data.loc[df_data['brand']=='hundai','brand'] = 'hyundai'
df_data.loc[df_data['brand']=='hinda','brand'] = 'honda'
df_data.loc[df_data['brand']=='nisan','brand'] = 'nissan'

In [17]:
df_data.brand.unique()

array(['toyota', 'ford', 'dodge', 'chevrolet', 'gmc', 'chrysler', 'kia',
       'buick', 'infiniti', 'mercedes-benz', 'jeep', 'bmw', 'cadillac',
       'hyundai', 'mazda', 'honda', 'heartland', 'jaguar', 'acura',
       'harley-davidson', 'audi', 'lincoln', 'lexus', 'nissan', 'land',
       'maserati', 'peterbilt', 'ram'], dtype=object)

### Colors

In [18]:
df_data.color.unique()

array(['black', 'silver', 'blue', 'red', 'white', 'gray', 'orange',
       'brown', nan, 'no_color', 'gold', 'charcoal', 'turquoise', 'beige',
       'green', 'dark blue', 'maroon', 'phantom black', 'yellow',
       'color:', 'light blue', 'toreador red', 'bright white clearcoat',
       'billet silver metallic clearcoat', 'black clearcoat',
       'jazz blue pearlcoat', 'purple',
       'ruby red metallic tinted clearcoat', 'triple yellow tri-coat',
       'competition orange', 'off-white', 'shadow black',
       'magnetic metallic', 'ingot silver metallic', 'ruby red',
       'royal crimson metallic tinted clearcoat', 'kona blue metallic',
       'oxford white', 'lightning blue', 'ingot silver',
       'white platinum tri-coat metallic', 'guard',
       'tuxedo black metallic', 'tan', 'burgundy', 'super black',
       'cayenne red', 'morningsky blue', 'pearl white', 'glacier white'],
      dtype=object)

Let us simplify the colors to the basic colors: ['blue', 'black', 'yellow', 'red', 'white', 'silver', 'orange']. All other colors (including null values) will be categorized as 'others'.

In [19]:
basic_colors = ['blue', 'black', 'yellow', 'red', 'white', 'silver', 'orange']

In [20]:
def replace_color(str_color, basic_colors=basic_colors):
    for color in basic_colors:
        if color in str_color:
            return color
    return 'others'

In [21]:
df_data['color'] = df_data['color'].apply(lambda x:replace_color(str(x)))

In [22]:
df_data['color'].unique()

array(['black', 'silver', 'blue', 'red', 'white', 'others', 'orange',
       'yellow'], dtype=object)

### Manipulate 'condition' column. Convert everything to minutes.

In [23]:
df_data['condition'].unique()

array(['10 days left', '6 days left', '2 days left', '22 hours left',
       '20 hours left', '19 hours left', '3 days left', '21 hours left',
       '17 hours left', '2 hours left', '3 hours left', '34 minutes',
       '16 hours left', '18 hours left', '1 days left', '32 minutes',
       '14 hours left', '5 hours left', '4 days left', '9 days left',
       '23 hours left', '8 days left', '7 days left', '5 days left',
       '9 minutes', '1 minutes', '7 hours left', '16 minutes',
       '6 hours left', '1 hours left', 'Listing Expired', '13 days left',
       '24 hours left', '15 hours left', '53 minutes', '27 minutes',
       '12 days left', '15 days left', '30 minutes', '29 minutes',
       '28 minutes', '48 minutes', '11 days left', '4 hours left',
       '47 minutes', '12 hours left', '36 minutes'], dtype=object)

In [24]:
def convert_to_mins(condition):
    def str_to_num(strnum, cnt_err):
        try:
            num = int(lt_words[0])
            return num, cnt_err
        except ValueError as ex:
            cnt_err += 1
            return None, cnt_err
    cnt_err = 0
    if condition:
        temp = 0
        mins = 0
        
        lt_words = re.findall(r'(\w+)', condition)
        if lt_words:
            if len(lt_words)>=2:
                if lt_words[1].strip() == 'days':
                    temp, cnt_err = str_to_num(lt_words[0].strip(),cnt_err)
                    mins = temp*24*60
                elif lt_words[1].strip() == 'hours':
                    temp, cnt_err = str_to_num(lt_words[0].strip(),cnt_err)
                    mins = temp*60
                elif lt_words[1].strip() == 'minutes':
                    mins, cnt_err = str_to_num(lt_words[0],cnt_err)
                elif lt_words[1] == 'Expired':
                    mins = 0
        return mins
    return None


In [25]:
df_data['condition'] = df_data['condition'].apply(lambda x:convert_to_mins(x))

In [26]:
df_data['condition'].unique()

array([14400,  8640,  2880,  1320,  1200,  1140,  4320,  1260,  1020,
         120,   180,    34,   960,  1080,  1440,    32,   840,   300,
        5760, 12960,  1380, 11520, 10080,  7200,     9,     1,   420,
          16,   360,    60,     0, 18720,   900,    53,    27, 17280,
       21600,    30,    29,    28,    48, 15840,   240,    47,   720,
          36], dtype=int64)

# Cleaned data

In [27]:
df_data.head()

,price,brand,model,year,title_status,mileage,color,vin,lot,state,country,condition
0,6300,toyota,cruiser,2008,clean vehicle,274117.0,black,jtezu11f88k007763,159348797,new jersey,usa,14400
1,2899,ford,se,2011,clean vehicle,NaN,silver,2fmdk3gc4bbb02217,166951262,tennessee,usa,8640
2,5350,dodge,mpv,2018,clean vehicle,39590.0,silver,3c4pdcgg5jt346413,167655728,georgia,usa,2880
3,25000,ford,door,2014,clean vehicle,64146.0,blue,1ftfw1et4efc23745,167753855,virginia,usa,1320
4,27700,chevrolet,1500,2018,clean vehicle,6654.0,red,3gcpcrec2jg473991,167763266,florida,usa,1320


In [28]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 12 columns):
price           2499 non-null int32
brand           2499 non-null object
model           2499 non-null object
year            2499 non-null int32
title_status    2499 non-null object
mileage         2309 non-null float64
color           2499 non-null object
vin             2499 non-null object
lot             2499 non-null int32
state           2499 non-null object
country         2499 non-null object
condition       2499 non-null int64
dtypes: float64(1), int32(3), int64(1), object(7)
memory usage: 205.1+ KB


## Convert to '.csv' and save the data.

In [29]:
df_data.to_csv('cleaned_data.csv', index=False)